In [ ]:
%%html
<!-- Improve the styling of the Notebook. -->
<link href="https://fonts.googleapis.com/css2?family=Source+Code+Pro&family=Source+Sans+3&family=Source+Serif+4:opsz@8..60&display=swap" rel="stylesheet">
<style>
   div.jp-MarkdownOutput p { font-family: 'Source Serif 4', serif; width: 50em; }
   div.jp-MarkdownOutput h1,h2,h3,h4,h5,h6 { font-family: 'Source Sans 3', sans-serif; }
   div.cm-line { font-family: 'Source Code Pro', monospace; }
</style>

In [ ]:
import hail as hl

# Importing a VCF File as a Hail Matrix Table

Matrix tables are a unique feature of Hail that are missing in other distributed, partitioned dataframe systems. Matrix tables were inspired by the VCF format which represents one or more genomic sequences. Each row is a genomic locus, like "chr1:123". Each column is a sample identified by a string of characters and numbers, like "NA12345". 

[`hl.import_vcf`](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) imports a VCF file as a Hail Matrix Table.

In [ ]:
mt = hl.import_vcf('data/sample.vcf', reference_genome='GRCh38', min_partitions=2)

In [ ]:
mt

# Showing the Row, Column, and Entry Fields

Matrix tables, just like Tables, are recipes. Their printed form provides no useful information. [`MatrixTable.show`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.show), which is an action, displays the entry field values of the first few rows and columns of the matrix.

In [ ]:
mt.show(n_rows=3, n_cols=3)

This is the top-left corner of this matrix table.

Each column represents a sample and is shown with its sample identifier: "Sample1", "Sample2", and "Sample3". Each row represents a variant and is shown with the variant's locus and alleles. Each entry represents a sequenced genotype. This sequenced genotype comprises four fields: the genotype call "GT", the total depth "DP", the phred-scaled genotype likelihoods "PL", and the per-allele depth "AD". See the [VCF Specification version 4.3](https://samtools.github.io/hts-specs/VCFv4.3.pdf) for details.

Seven fields are visible: two row fields: locus and alleles; four entry fields: GT, DP, PL, and AD; one column field: s.

[`MatrixTable.show`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.show) only shows the entry fields, the row key fields, and the column key fields. The matrix table usually has other row and column fields that are not displayed by show. [`MatrixTable.describe`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.describe) lists all the fields.

In [ ]:
mt.describe()

[`MatrixTable.rows`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.rows) returns a Hail table with all the row fields. [`MatrixTable.cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.cols) returns a Hail table with all the column fields. We can, of course, use [`Table.describe`]((https://hail.is/docs/0.2/hail.Table.html#hail.Table.describe) and [`Table.show`]((https://hail.is/docs/0.2/hail.Table.html#hail.Table.show) to interrogate either table.

⚠️⚠️⚠️ Confusing Behavior Alert ⚠️⚠️⚠️

Matrix table columns are ordered in the same way as they are in the imported VCF. In contrast, every Hail table, including the `mt.cols()` table, is _always_ ordered by its key field.

This is indeed confusing; however, it is a necessary comprimise to avoid sorting, at great cost, the columns of a VCF.

In [ ]:
mt.cols().show(n=3)

We can preserve the ordering of the cols table by removing the setting the column key to the empty key (which requires no particular ordering).

In [ ]:
mt.key_cols_by().cols().show(n=3)

In [ ]:
mt.rows().show(n=3)

"info.DP" is our first example of a _nested_ field. The "info" field contains a "DP" field, the sum total depth across all sample. There are many ways to access a nested field:

In [ ]:
mt.info.DP
mt['info'].DP
mt.info['DP']
mt['info']['DP']

### Exercise

It's also possible to show individual fields. Try showing the info.DP field.

# Adding Row, Column, and Entry Fields with Annotate

## Row Fields

These "sum total depths" look fishy: they're too small. Let's compute the actual sum with [`hl.agg.sum`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.sum) and add it as a new row field with [`MatrixTable.annotate_rows`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows).

In [ ]:
mt = mt.annotate_rows(
    the_actual_sum_total_DP = hl.agg.sum(mt.DP)
)
mt.rows().show(n=3)

Not even close to correct! Let's replace the "info.DP" value with the actual sum. Notice that we use [`StructExpression.annotate`](https://hail.is/docs/0.2/hail.expr.StructExpression.html#hail.expr.StructExpression.annotate) to add a new field to the "info" field.

In [ ]:
mt = mt.annotate_rows(
    info = mt.info.annotate(
        DP = hl.agg.sum(mt.DP)
    )
)
mt.rows().show(n=3)

## Column Fields

Hail has an extensive [library of random functions](https://hail.is/docs/0.2/functions/random.html) as well as a [library of statistical distributions and tests](https://hail.is/docs/0.2/functions/stats.html). Let's use [`MatrixTable.annotate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) to randomly generate a height field for each sample.

In [ ]:
mt = mt.annotate_cols(
    height_ft = hl.rand_norm(5 + 8/12, 2/12)
)
mt.key_cols_by().cols().show(n=3)

## Entry Fields

[`MatrixTable.annotate_entries`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_entries) adds new entry fields. We can show just one field (with its relevant keys) using [`Expression.show`](https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.show).

In [ ]:
mt = mt.annotate_entries(
    low_DP = mt.DP < 15
)
mt.low_DP.show(n_rows=3, n_cols=3)

[`Expression.show`](https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.show) also works with compound expressions, such as a struct expression. A struct expression combines multiple values into one struct value.

In [ ]:
hl.struct(low_DP=mt.low_DP, DP=mt.DP, GT=mt.GT).show(n_rows=3, n_cols=3)

### Exercise

Add an entry field which is the sum of the AD array. See [collection functions](https://hail.is/docs/0.2/functions/collections.html).

# Filtering Rows, Columns, and Entries

[`MatrixTable.filter_rows`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_rows), [`MatrixTable.filter_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_cols), and [`MatrixTable.filter_entries`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_entries) respectively remove rows, columns, and entries from a matrix table.

We make a copy of our recipe as `xx` so that we can return to the full dataset later.

In [ ]:
xx = mt

In [ ]:
xx = xx.filter_rows(
    xx.locus.contig != 'chr1'
)
xx.show(n_rows=3, n_cols=2)

In [ ]:
xx = xx.filter_cols(
    xx.s > 'Sample4'
)
xx.show(n_rows=3, n_cols=2)

A filtered entry is like a hole in the matrix. The other entries in a row or column are still present, so Hail still treats that row and that column as part of the dataset; however, the filtered entry itself is shown as if all its entry fields are missing.

In [ ]:
xx = xx.filter_entries(
    xx.GT.is_het()
)
xx.show(n_rows=3, n_cols=2)

### Exercise

Filter to rows in chromosome 2.

# Head and Tail of the Dataset

[`MatrixTable.head`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.head) and [`MatrixTable.tail`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.tail) filter the dataset to corners of the matrix.

In [ ]:
mt.head(n_rows=3, n_cols=3).GT.show()

There is currently a bug in `tail`: it calls `n_rows` `n`. This will be fixed in 0.2.121.

In [ ]:
mt.tail(n=3, n_cols=3).GT.show()

Head and tail can be combined to filter to the top-right or bottom-left corners of the matrix.

In [ ]:
mt.head(n_rows=3, n_cols=None).tail(n=None, n_cols=3).GT.show()

# Aggregating Rows, Columns, and Entries

[`MatrixTable.aggregate_entries`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_entries) aggregates the entire dataset into a single Python value. [`hl.agg.group_by`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.group_by) partitions values into groups and aggregates each group separately.

In [ ]:
mt.aggregate_entries(
    hl.agg.group_by(mt.GT, hl.agg.count())
)

[`MatrixTable.aggregate_rows`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_rows) and [`MatrixTable.aggregate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) respectively aggregate all the row fields or all the column fields into a single Python value.

[`hl.agg.stats`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats) computes the mean, standard deviation, min, max, count, and sum of a numeric field.

In [ ]:
mt.aggregate_rows(
    hl.agg.stats(mt.info.DP)
)

In [ ]:
mt.aggregate_cols(
    hl.agg.stats(mt.height_ft)
)

[`MatrixTable.annotate_rows`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows) permits aggregating the entries of each row separately. This produces a single Hail value for each row which is necessarily stored in a row field. [`CallExpression.n_alt_alleles`](https://hail.is/docs/0.2/hail.expr.CallExpression.html#hail.expr.CallExpression.n_alt_alleles) returns the number of alternate alleles in the genotype call. For example, `0/0` has zero alternate alleles and `1/1` has two.

In [ ]:
mt.annotate_rows(
    alternate_allele_frequency = hl.agg.mean(mt.GT.n_alt_alleles()) / 2.0
).alternate_allele_frequency.show(n=10)

[`MatrixTable.annotate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) likewise permits aggregating each column of entries.

In [ ]:
mt.annotate_cols(
    mean_sample_depth = hl.agg.mean(mt.DP)
).key_cols_by().cols().select('s', 'mean_sample_depth').show(n=10)

### Exercise

Filter to rows with more homozygous reference calls than heterozygous calls.

### Exercise

Filter to samples whose mean depth across all variants is greater than 18.

# Aggregating within Groups of Rows or Groups of Columns

[`MatrixTable.group_rows_by`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.group_rows_by) aggregates groups of rows to produce a new matrix table whose rows are the groups. 

In [ ]:
mt.group_rows_by(
    contig=mt.locus.contig
).aggregate(
    n_alt_alleles = hl.agg.sum(mt.GT.n_alt_alleles())
).show(n_rows=3, n_cols=10)

[`MatrixTable.group_cols_by`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.group_cols_by) is the column analogue.

In [ ]:
mt.group_cols_by(
    is_shorter_than_5_8 = mt.height_ft < (5 + 8/12)
).aggregate(
    n_alt_alleles = hl.agg.sum(mt.GT.n_alt_alleles())
).show(n_rows=10, n_cols=2)

### Exercise

Calculate the mean depth for each contig.

# Writing and Reading Matrix Tables in Hail Native Format

Hail has a partitioned, indexed, binary file format for quickly reading and writing matrix tables. [`MatrixTable.write`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.write) is the action which writes a matrix table in Hail native format. We use the ".mt" file extension by convention.

In [ ]:
mt.write('output/sample_vcf.mt')

Unless you're using BGEN, a binary format for which Hail has excellent support, you should always read from a Hail native format file instead of importing.

[`hl.read_matrix_table`](https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_matrix_table) reads matrix tables stored in Hail native format.

In [ ]:
mt = hl.read_matrix_table('output/sample_vcf.mt')

# Exporting a Matrix Table

A matrix table supports [export to many formats](https://hail.is/docs/0.2/methods/impex.html#export) including VCF, BGEN, and PLINK.

Always export VCFs using block GZIP compression and "header_per_shard" or "separate_header". VCF does not support boolean FORMAT fields so we must recode `low_DP` to an integer using [`hl.if_else`](https://hail.is/docs/0.2/functions/core.html#hail.expr.functions.if_else).

In [ ]:
xx = mt
xx = xx.annotate_entries(low_DP=hl.if_else(xx.low_DP, 1, 0))
hl.export_vcf(xx, 'output/sample_vcf.vcf.bgz', parallel='header_per_shard')

In [ ]:
!ls output/sample_vcf.vcf.bgz/

⚠️⚠️⚠️ Confusing Behavior Alert ⚠️⚠️⚠️

BGEN datasets are usually two files: a .bgen file and a .sample file. [`hl.export_bgen`](https://hail.is/docs/0.2/methods/impex.html#hail.methods.export_bgen) expects a file path _without_ an extension. A file named `....sample` contains the sample IDs. A file or folder named `....bgen` contains the genotype probabilities in BGEN format.

In [ ]:
xx = mt
xx = xx.annotate_entries(
    GP=(hl.case()
        .when(mt.GT.is_hom_ref(), [1, 0, 0])
        .when(mt.GT.is_het(), [0, 1, 0])
        .when(mt.GT.is_hom_var(), [0, 0, 1])
        .or_error(hl.format('Unexpected GT: %s', mt.GT))
       )
)
hl.export_bgen(xx, 'output/sample_vcf', gp=xx.GP, parallel='header_per_shard')

In [ ]:
!ls output/sample_vcf.bgen/

In [ ]:
!head -n 4 output/sample_vcf.sample

# Collecting a Matrix Table

Matrix tables do not support `collect` because there is no obvious Python analogue to the matrix table. A list of list or a NumPy matrix both seem reasonable. Matrix table does not support `to_pandas` because Pandas DataFrames have a large per-column overhead and most matrix tables have many columns, each with many entry fields.

Instead, matrix tables provide methods for producing tables which can be converted to lists or Pandas DataFrames.

[`MatrixTable.make_table`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.make_table) creates a table with one field for every column for every entry field.

In [ ]:
xx = mt
xx = xx.make_table()
xx.show(n=3)

In [ ]:
xx.to_pandas()

[`MatrixTable.localize_entries`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.localize_entries) is a confusingly named method which converts the entries into array row fields:

In [ ]:
xx = mt
xx = xx.localize_entries('entries', 'columns')
xx.show()

In [ ]:
xx.collect()

Hail incorrectly converts this table into a Pandas DataFrame (notice the entries are the field names, not the field values). This is a [known bug](https://github.com/hail-is/hail/issues/13512) which will be fixed in a future version of Hail.

In [ ]:
xx.to_pandas()